# Historical Event Narrator - Training Notebook (Colab)

This notebook is designed to run on Google Colab (Free Tier T4 GPU).
It clones the repository, installs dependencies, and runs the LoRA fine-tuning.

In [ ]:
# 1. Clone Repository
!git clone https://github.com/Keerthirajan58/Historical-Event-Narrator.git
%cd Historical-Event-Narrator
!git checkout keerthi-dev

In [ ]:
# 2. Install Dependencies
!pip install -r requirements.txt
!pip install bitsandbytes accelerate peft transformers datasets

In [ ]:
# 3. Run Training
# We use the synthetic data we generated (ensure it's in the repo or upload it)
# If data is not in repo (because it's large), you might need to run preprocess here or upload it.
# Since we pushed the 50-example sample, we can run a test train.

from src.train import train

# Run training with Mistral-7B (requires GPU)
# Note: On Colab T4, we use 4-bit quantization
train(
    train_file="data/processed/train.jsonl",
    val_file="data/processed/test.jsonl",
    model_name="mistralai/Mistral-7B-v0.1",
    num_epochs=1,
    batch_size=4,
    use_quantization=True
)

In [ ]:
# 4. Inference Test
from src.model import get_model_and_tokenizer, get_peft_model_wrapper
from peft import PeftModel
import torch

# Load Base Model
base_model, tokenizer = get_model_and_tokenizer("mistralai/Mistral-7B-v0.1", use_quantization=True)

# Load Adapter
model = PeftModel.from_pretrained(base_model, "./results")

prompt = "Narrate the historical event with a creative twist.\n\nTopic: The Moon Landing\n\nNarrative:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))